<a href="https://colab.research.google.com/github/sangkonpark/1day-1commit/blob/main/4%EC%A3%BC%EC%B0%A8_%EC%88%99%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 448 kB 43.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
train_df = pd.read_csv('train.csv', index_col='index')

In [9]:
train_df.head()

,category,data
index,,
0,2,신혼부부위한 주택정책 보다 보육시설 늘려주세요.. 국민세금으로 일부를 위한 정책펴지...
1,0,학교이름에 '남자'도 붙여주세요. 울산여자중학교에 재학중인 학생입니다 최근 양성평등...
2,1,"빙상연맹, 대한축구협회등 각종 체육협회의 비리를 철저하게 밝혀주세요.. 최근 동계올..."
3,1,"티비 12세,15세 관람가도 연령확인 의무화 하자.. 제기 에전에 티비를 보다가 잠..."
4,1,무더운 여름철엔 남성들도 시원한 자율복장을 해야. 무더운 여름철에는 남성들도 노넥타...


In [10]:
len(train_df)

40000

In [11]:
train_df.isnull().sum()

category    0
data        8
dtype: int64

In [12]:
# 결측치가 있는 행 전부 삭제
train_df = train_df.dropna(axis = 0, how = 'any')

In [13]:
len(train_df)

39992

In [14]:
# 형태소 분석기를 통해 형태소 단위로 추출하는 코드
from konlpy.tag import Okt

In [15]:
tokenizer = Okt()

In [16]:
# 전처리를 하는 코드

def preprocessing(data):
  train_data = []
  for sentence in data:
		# tokenizer.morphs()는 형태소 추출 기능입니다.
    temp_data = tokenizer.morphs(sentence) # 토큰화
    train_data.append(temp_data)
  train_data = list(map(' '.join, train_data))
  return train_data

In [17]:
# 데이터프레임의 data 열에 대해서 해당 함수를 통해 전처리 진행

train_data = preprocessing(train_df['data'])

In [18]:
# train_df를 훈련 데이터와 테스트 데이터로 분리 후, TF-IDF 행렬을 만들고 나이브 베이즈 분류기 수행

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [19]:
# train_df_split을 사용하여 train_df['category']데이터를 1.7:2.5 비율로 분리하여 훈련 데이터와 테스트 데이터 만들기
# 변수명은 X-train, y_train, X-test, y_test를 사용

X_train, X_test, y_train, y_test = train_test_split(train_data, train_df['category'], random_state = 0)

In [21]:
# 훈련 데이터 X-train을 TF-IDF 행렬로 변환

count_vect = CountVectorizer(max_features=10000)
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [24]:
# X_test에 대해서도 TF-IDF 행렬로 변환

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [25]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [26]:
new_sent = train_df.iloc[2]['data']

In [27]:
new_sent = preprocessing([new_sent])

In [28]:
y_pred = clf.predict(tfidf_vectorizer(X_test))

In [29]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3372
           1       0.90      0.89      0.90      3296
           2       0.88      0.85      0.86      3330

    accuracy                           0.86      9998
   macro avg       0.86      0.86      0.86      9998
weighted avg       0.86      0.86      0.86      9998

